In [ ]:
import zfit
import tensorflow as tf
import pandas as pd
import numpy as np

class p5_prime_pdf(zfit.pdf.ZPDF):
    _N_OBS = 3  # dimension, can be omitted
    _PARAMS = ['param_fl','param_s3','param_p5']  # the name of the parameters

    def _unnormalized_pdf(self, x):
        x, y, z  = x.unstack_x()  # x=costk, y=costl, z=phib
        param_fl = self.params['param_fl']
        param_s3 = self.params['param_s3']
        param_p5 = self.params['param_p5']
        
        return (9/(8*np.pi))*((3/4)*(1-param_fl)* (1-x**2) + param_fl * x**2 + \
               (1/4) *(1 - param_fl)* (1-x**2)* (2 * y**2 - 1)- \
               param_fl * x**2 * (2 * y**2 - 1) + param_s3 *(1-x**2)*(1-y**2) * tf.cos(2*z)+ \
               tf.sqrt(param_fl*(1-param_fl)) * param_p5 * 2*tf.sqrt(1-x**2) *x * tf.sqrt(1-y**2)* tf.cos(z))



def p5p_3D_fit(cost_k_data,cost_l_data, phi_b_data,param_fl_start,param_s3_start,param_p5_start,weight,floating=True):

    # create space
    obs_cost_k  = zfit.Space('cosTheta_K', limits=(min(cost_k_data),max(cost_k_data)))
    obs_cost_l  = zfit.Space('cosTheta_L', limits=(min(cost_l_data),max(cost_l_data)))
    obs_phi_b   = zfit.Space('phi_B', limits=(0, np.pi))

    obs = obs_cost_k * obs_cost_l * obs_phi_b

        # data
    df_sig         = pd.concat([cost_k_data,cost_l_data, phi_b_data],  axis=1)
    data_fit_sig   = zfit.Data.from_pandas(df_sig, obs = obs,weights=weight)

        #parameter variable
    r = np.round(np.random.rand(),decimals=8)

        #parameters
    param_fl  = zfit.Parameter(f'param_fl_{r}',  param_fl_start,  floating=floating)
    param_s3  = zfit.Parameter(f'param_s3_{r}',  param_s3_start,  floating=floating)
    param_p5  = zfit.Parameter(f'param_p5_{r}',  param_p5_start,  floating=floating)

    extended = zfit.Parameter(f"extended_{r}", 10000)

    pdf_3D = p5_prime_pdf(param_fl=param_fl, param_s3=param_s3, param_p5=param_p5, obs=obs, extended=extended)
    pdf_3D.update_integration_options(max_draws=10000, tol=1e-4)  # Adjust parameters as needed


        # create NLL
    nll       = zfit.loss.ExtendedUnbinnedNLL(model=pdf_3D, data=data_fit_sig)
    minimizer = zfit.minimize.Minuit()
    result    = minimizer.minimize(nll)

    if floating==True:
        result.hesse(name='hesse')
               
    print(result.params)
    print("Fit result valid:")
    print(result.valid)
    
    return pdf_3D

def fold_data(data):
    data['cosTheta_L'] = data['cosTheta_L'].abs()
    phi_b_gr = data.query(f"phi_B > {np.pi}")
    phi_b_gr['phi_B'] = 2 * np.pi - phi_b_gr['phi_B']
    phi_b_kl = data.query(f"phi_B <= {np.pi}")
    folded_data = pd.concat([phi_b_kl, phi_b_gr])
    return folded_data

mc_ele_only_signal_fold = fold_data(mc_ele_only_signal)
mc_ele_only_signal_wo_klcut_fold = fold_data(mc_ele_only_signal_wo_klcut)


mc_ele_only_signal_weight_fold=mc_ele_only_signal_fold["PIDCorrectionAndAcceptanceWeight"]

mc_ele_only_signal_weight_wo_klcut_fold=mc_ele_only_signal_wo_klcut_fold["PIDCorrectionAndAcceptanceWeight"]



# Example usage with adjusted starting parameters
param_fl_start = 0.5  # Adjust based on your data 0.5
param_s3_start = 0.01  # Adjust based on your data -0.01
param_p5_start = 0.02  # Adjust based on your data 0.02

# Fit with phi_B greater than np.pi
print("Fitting for phi_B greater than pi:")
pdf_fit_signal_gr = p5p_3D_fit(mc_ele_only_signal_fold["cosTheta_K"], mc_ele_only_signal_fold["cosTheta_L"], mc_ele_only_signal_fold["phi_B"],
                               param_fl_start, param_s3_start, param_p5_start, weight=mc_ele_only_signal_weight_fold,
                               floating=True)

# Fit with phi_B less than or equal to np.pi
print("Fitting for phi_B less than or equal to pi:")
pdf_fit_signal_kl = p5p_3D_fit(mc_ele_only_signal_wo_klcut_fold["cosTheta_K"], mc_ele_only_signal_wo_klcut_fold["cosTheta_L"], mc_ele_only_signal_wo_klcut_fold["phi_B"],
                               param_fl_start, param_s3_start, param_p5_start, weight=mc_ele_only_signal_weight_wo_klcut_fold,
                               floating=True)


# Calculate total signal events
mc_ele_only_signal_MC_data = sum(mc_ele_only_signal_weight_fold)
mc_ele_only_signal_MC_data_wo_klcut = sum(mc_ele_only_signal_weight_wo_klcut_fold)

# Now, let's print the total signal events again
print("Signal Events in MC Data =", mc_ele_only_signal_MC_data)
print("Signal Events in MC Data_wo_klcut =", mc_ele_only_signal_MC_data_wo_klcut)


## After angular cut

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.integrate import simps

# Define the ranges for each variable
cosTheta_K_range = np.linspace(-0.9, 1.0, 100)
cosTheta_L_range = np.linspace(0, 1.0, 100)
phi_B_range = np.linspace(0, np.pi, 100)

# Create a meshgrid of points
cosTheta_K_grid, cosTheta_L_grid, phi_B_grid = np.meshgrid(cosTheta_K_range, cosTheta_L_range, phi_B_range)

# Flatten the grids to evaluate the PDF
points = np.vstack([cosTheta_K_grid.flatten(), cosTheta_L_grid.flatten(), phi_B_grid.flatten()]).T

# Evaluate the fitted PDF at each point
pdf_values_gr = pdf_fit_signal_gr.pdf(points).numpy().reshape(cosTheta_K_grid.shape)

# Integration step
# Integrate the PDF over cosTheta_L and phi_B for each cosTheta_K
integrated_pdf_values_K = simps(simps(pdf_values_gr, phi_B_range, axis=2), cosTheta_L_range, axis=1)

# Normalize the integrated PDF
pdf_values_K = integrated_pdf_values_K * sum(mc_ele_only_signal_weight_fold) * np.diff(cosTheta_K_range)[0]

# Plot the results
plt.figure(figsize=(8, 6))
n, bins, _ = plt.hist(mc_ele_only_signal_fold["cosTheta_L"], bins=100, weights=mc_ele_only_signal_weight_fold, density=False, alpha=0.5, label='Observed data')
bin_centers = (bins[:-1] + bins[1:]) / 2
# Calculate errors
bin_weights = [mc_ele_only_signal_weight_fold.iloc[np.where((mc_ele_only_signal_fold["cosTheta_L"] >= bins[i]) & (mc_ele_only_signal_fold["cosTheta_L"] < bins[i+1]))[0]].tolist() for i in range(len(bins) - 1)]
errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]
plt.errorbar(bin_centers, n, yerr=errors, fmt='o', color='black', label='Error Bars')
plt.plot(cosTheta_L_range, pdf_values_K, 'r', label='Fit')
plt.xlabel('cosTheta_L')
plt.ylabel('Counts')
plt.legend()
plt.title('Projection along cosTheta_L')
plt.grid(True)

# Repeat similar steps for cosTheta_L and phi_B
# For cosTheta_L
integrated_pdf_values_L = simps(simps(pdf_values_gr, phi_B_range, axis=2), cosTheta_K_range, axis=0)
pdf_values_L = integrated_pdf_values_L * sum(mc_ele_only_signal_weight_fold) * np.diff(cosTheta_L_range)[0]

plt.figure(figsize=(8, 6))
n, bins, _ = plt.hist(mc_ele_only_signal_fold["cosTheta_K"], bins=100, weights=mc_ele_only_signal_weight_fold, density=False, alpha=0.5, label='Observed data')
bin_centers = (bins[:-1] + bins[1:]) / 2
bin_weights = [mc_ele_only_signal_weight_fold.iloc[np.where((mc_ele_only_signal_fold["cosTheta_K"] >= bins[i]) & (mc_ele_only_signal_fold["cosTheta_K"] < bins[i+1]))[0]].tolist() for i in range(len(bins) - 1)]
errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]
plt.errorbar(bin_centers, n, yerr=errors, fmt='o', color='black', label='Error Bars')
plt.plot(cosTheta_K_range, pdf_values_L, 'r', label='Fit')
plt.xlabel('cosTheta_K')
plt.xlim(-1.25,1.25)
plt.ylabel('Counts')
plt.legend()
plt.title('Projection along cosTheta_K')
plt.grid(True)

# For phi_B
integrated_pdf_values_B = simps(simps(pdf_values_gr, cosTheta_L_range, axis=1), cosTheta_K_range, axis=0)
pdf_values_B = integrated_pdf_values_B * sum(mc_ele_only_signal_weight_fold) * np.diff(phi_B_range)[0]

plt.figure(figsize=(8, 6))
n, bins, _ = plt.hist(mc_ele_only_signal_fold["phi_B"], bins=100, weights=mc_ele_only_signal_weight_fold, density=False, alpha=0.5, label='Observed data')
bin_centers = (bins[:-1] + bins[1:]) / 2
bin_weights = [mc_ele_only_signal_weight_fold.iloc[np.where((mc_ele_only_signal_fold["phi_B"] >= bins[i]) & (mc_ele_only_signal_fold["phi_B"] < bins[i+1]))[0]].tolist() for i in range(len(bins) - 1)]
errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]
plt.errorbar(bin_centers, n, yerr=errors, fmt='o', color='black', label='Error Bars')
plt.plot(phi_B_range, pdf_values_B, 'r', label='Fit')
plt.xlabel('phi_B')
plt.ylabel('Counts')
plt.legend()
plt.title('Projection along phi_B')
plt.grid(True)

plt.show()

# Integration step over the entire 3D space
integrated_pdf_total = simps(simps(simps(pdf_values_gr, phi_B_range, axis=2), cosTheta_L_range, axis=1), cosTheta_K_range, axis=0)

# Find the number of events by normalizing the total integrated PDF
num_events_total = integrated_pdf_total * sum(mc_ele_only_signal_weight_fold)

print("Total number of events:", num_events_total)

## Before angular cut

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.integrate import simps

# Define the ranges for each variable
cosTheta_K_range = np.linspace(-0.95, 1.0, 100)
cosTheta_L_range = np.linspace(0, 1.0, 100)
phi_B_range = np.linspace(0, np.pi, 100)

# Create a meshgrid of points
cosTheta_K_grid, cosTheta_L_grid, phi_B_grid = np.meshgrid(cosTheta_K_range, cosTheta_L_range, phi_B_range)

# Flatten the grids to evaluate the PDF
points = np.vstack([cosTheta_K_grid.flatten(), cosTheta_L_grid.flatten(), phi_B_grid.flatten()]).T

# Evaluate the fitted PDF at each point
pdf_values_gr = pdf_fit_signal_kl.pdf(points).numpy().reshape(cosTheta_K_grid.shape)

# Integration step
# Integrate the PDF over cosTheta_L and phi_B for each cosTheta_K
integrated_pdf_values_K = simps(simps(pdf_values_gr, phi_B_range, axis=2), cosTheta_L_range, axis=1)

# Normalize the integrated PDF
pdf_values_K = integrated_pdf_values_K * sum(mc_ele_only_signal_weight_wo_klcut_fold) * np.diff(cosTheta_K_range)[0]

# Plot the results
plt.figure(figsize=(8, 6))
n, bins, _ = plt.hist(mc_ele_only_signal_wo_klcut_fold["cosTheta_L"], bins=100, weights=mc_ele_only_signal_weight_wo_klcut_fold, density=False, alpha=0.5, label='Observed data')
bin_centers = (bins[:-1] + bins[1:]) / 2
# Calculate errors
bin_weights = [mc_ele_only_signal_weight_wo_klcut_fold.iloc[np.where((mc_ele_only_signal_wo_klcut_fold["cosTheta_L"] >= bins[i]) & (mc_ele_only_signal_wo_klcut_fold["cosTheta_L"] < bins[i+1]))[0]].tolist() for i in range(len(bins) - 1)]
errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]
plt.errorbar(bin_centers, n, yerr=errors, fmt='o', color='black', label='Error Bars')
plt.plot(cosTheta_L_range, pdf_values_K, 'r', label='Fit')
plt.xlabel('cosTheta_L')
plt.ylabel('Counts')
plt.legend()
plt.title('Projection along cosTheta_L')
plt.grid(True)

# Repeat similar steps for cosTheta_L and phi_B
# For cosTheta_L
integrated_pdf_values_L = simps(simps(pdf_values_gr, phi_B_range, axis=2), cosTheta_K_range, axis=0)
pdf_values_L = integrated_pdf_values_L * sum(mc_ele_only_signal_weight_wo_klcut_fold) * np.diff(cosTheta_L_range)[0]

plt.figure(figsize=(8, 6))
n, bins, _ = plt.hist(mc_ele_only_signal_wo_klcut_fold["cosTheta_K"], bins=100, weights=mc_ele_only_signal_weight_wo_klcut_fold, density=False, alpha=0.5, label='Observed data')
bin_centers = (bins[:-1] + bins[1:]) / 2
bin_weights = [mc_ele_only_signal_weight_wo_klcut_fold.iloc[np.where((mc_ele_only_signal_wo_klcut_fold["cosTheta_K"] >= bins[i]) & (mc_ele_only_signal_wo_klcut_fold["cosTheta_K"] < bins[i+1]))[0]].tolist() for i in range(len(bins) - 1)]
errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]
plt.errorbar(bin_centers, n, yerr=errors, fmt='o', color='black', label='Error Bars')
plt.plot(cosTheta_K_range, pdf_values_L, 'r', label='Fit')
plt.xlabel('cosTheta_K')
plt.xlim(-1.25,1.25)
plt.ylabel('Counts')
plt.legend()
plt.title('Projection along cosTheta_K')
plt.grid(True)

# For phi_B
integrated_pdf_values_B = simps(simps(pdf_values_gr, cosTheta_L_range, axis=1), cosTheta_K_range, axis=0)
pdf_values_B = integrated_pdf_values_B * sum(mc_ele_only_signal_weight_wo_klcut_fold) * np.diff(phi_B_range)[0]

plt.figure(figsize=(8, 6))
n, bins, _ = plt.hist(mc_ele_only_signal_wo_klcut_fold["phi_B"], bins=100, weights=mc_ele_only_signal_weight_wo_klcut_fold, density=False, alpha=0.5, label='Observed data')
bin_centers = (bins[:-1] + bins[1:]) / 2
bin_weights = [mc_ele_only_signal_weight_wo_klcut_fold.iloc[np.where((mc_ele_only_signal_wo_klcut_fold["phi_B"] >= bins[i]) & (mc_ele_only_signal_wo_klcut_fold["phi_B"] < bins[i+1]))[0]].tolist() for i in range(len(bins) - 1)]
errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]
plt.errorbar(bin_centers, n, yerr=errors, fmt='o', color='black', label='Error Bars')
plt.plot(phi_B_range, pdf_values_B, 'r', label='Fit')
plt.xlabel('phi_B')
plt.ylabel('Counts')
plt.legend()
plt.title('Projection along phi_B')
plt.grid(True)

plt.show()



# Integration step over the entire 3D space
integrated_pdf_total = simps(simps(simps(pdf_values_gr, phi_B_range, axis=2), cosTheta_L_range, axis=1), cosTheta_K_range, axis=0)

# Find the number of events by normalizing the total integrated PDF
num_events_total = integrated_pdf_total * sum(mc_ele_only_signal_weight_wo_klcut_fold)

print("Total number of events:", num_events_total)
